In [1]:
import dvc.api
import csv

In [2]:
genres_file = dvc.api.read(
    'data/data_by_genres_o.csv',
    repo='https://github.com/konradszewczyk/Spotify_DataMining'
)

CloneError: Failed to clone repo 'https://github.com/konradszewczyk/Spotify_DataMining' to '/tmp/tmpfnkzmcjkdvc-clone'

In [ ]:
print(genres_file)

In [ ]:
genres_csv = csv.reader(genres_file)